In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.utils import to_categorical
from numpy import argmax
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model


Using TensorFlow backend.


In [2]:
locstr = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/Neural-Net-Waves/Use_Conc/'

architecture = [(100,100)]

repo_string = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/WIFF-Model/'
training_data_string = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/Data/Neural_Net_Data/6-hourly-2009/Training_converged.mat'
network_save_string = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/Data/Neural_Net_Data/Networks/Classifier/'

misc_string = repo_string + 'Misc/NN_params.mat'
# These are for where to save "pickles" of the training/validation datasets. Should only create one on which any potential architectures or loss functions are trained against
pickle_string_class = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/Data/Neural_Net_Data/Training_Pickles/Use_Conc/Classifier/'


In [3]:
import h5py

# Here is imported as a matlab file. 
training_data = h5py.File(training_data_string, 'r')
misc_data = loadmat(misc_string)

In [4]:
input_vec = training_data['in'][()].T
output_vec = training_data['out'][()].T

Freq = misc_data['f']
dFreq = Freq * (np.sqrt(1.1) - np.sqrt(1/1.1)); 
Redge = misc_data['redge'].T
Rcent = misc_data['rcent'].T

In [5]:
ice_conc = np.expand_dims(input_vec[:,26],axis=1)
ice_thick = np.expand_dims(input_vec[:,25],axis=1)
wave_spec = input_vec[:,:25]

# wave_spec[wave_spec < 10**-8] = 0
wave_energy = np.sum(wave_spec*np.expand_dims(dFreq,axis=1).T,axis=1)
peak_loc = np.argmax(wave_spec,axis=1)
peak_freq = Freq[peak_loc]

num_frac = np.sum(output_vec,axis=1)
usable = num_frac > 0
print('Out of ' + str(len(num_frac)) + ', ' + str(sum(usable)) + ' should run SP-WIFF')

classifier_input = np.concatenate((wave_spec,ice_thick,ice_conc),axis=1) # input_vec[:,:29] # ,peak_freq]).T
classifier_output = to_categorical(usable)

Out of 5087436, 2660124 should run SP-WIFF


In [6]:
# Make sure we keep the training and test data the same at all times
import pickle

try:

    X_train = pickle.load( open( pickle_string_class + "X_train.p", "rb" ) )
    X_test = pickle.load( open( pickle_string_class + "X_test.p", "rb" ) )
    y_train = pickle.load( open(pickle_string_class + "y_train.p","rb"))
    y_test = pickle.load( open(pickle_string_class + "y_test.p","rb"))
    print('found previously segmented data')

except:
        
        print('exception in loading segmented data')
        print('Creating new pickle files')
        X_train, X_test, y_train, y_test = train_test_split(classifier_input,classifier_output,test_size=0.3,stratify=classifier_output)
        
        pickle.dump(X_train, open(pickle_string_class + "X_train.p", "wb" ) )
        pickle.dump(X_test, open(pickle_string_class + "X_test.p", "wb" ) )
        pickle.dump(y_train, open(pickle_string_class + "y_train.p", "wb" ) )
        pickle.dump(y_test, open(pickle_string_class + "y_test.p", "wb" ) )
        
    

found previously segmented data


In [ ]:
for structno in range(len(architecture)):

    struct = architecture[structno]
    
    print('Size is ' + str(struct))

    classifier = Sequential();
    #First Hidden Layer - 27 inputs are spectrum + thickness + concentration
    classifier.add(Dense(struct[0], activation='relu', kernel_initializer='GlorotNormal', input_dim=27));
    print('Adding layer with size ' + str(struct[0]))

    #Other Hidden Layers
    for i in range(len(struct)-1):
        print('Adding layer with size ' + str(struct[i+1]))
        classifier.add(Dense(struct[i+1], activation='relu', kernel_initializer='GlorotNormal'));

    classifier.add(Dense(2, activation='softmax', kernel_initializer='GlorotNormal'));

    #Output Layer
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy']);

    # Output name
    outstr = 'Class-'

    for i in range(len(struct)):
        outstr = outstr + str(struct[i]) + 'x'

    outstr = outstr[:-1]

    fname = network_save_string + outstr + '.h5'

    print(fname)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
    
    classifier.fit(X_train,y_train,validation_data = (X_test,y_test),batch_size=2048, epochs=1,callbacks = [es])
    
    classifier.save(fname)

In [10]:
net_string = repo_string + 'Classifier/Classifier_v1.h5'  
classifier = load_model(net_string)
thresh = 0.54

y_pred_class =classifier.predict(X_test)

In [11]:
y_true = y_test[:,1]
y_pred = y_pred_class[:,1] > thresh

cm = confusion_matrix(y_true, y_pred)
print(cm)

tn, fp, fn, tp = cm.ravel()

fnr = fn / [fn + tp]
fpr = fp / [fp + tn]

[[651360  76834]
 [111686 686351]]


In [12]:
print('false positive rate is ' + str(round(100*fpr[0],2)) + '%')
print('false negative rate is ' + str(round(100*fnr[0],2)) + '%')

false positive rate is 10.55%
false negative rate is 14.0%


In [ ]:
# Find results for each threshold

nthresh = 500

thresher = np.linspace(0,1,nthresh)
fnr = np.zeros(nthresh)
fpr = np.zeros(nthresh)
toter = np.zeros(nthresh)
reduc = np.zeros(nthresh)
y_true = y_test[:,1]

for i in range(nthresh):
    
    y_pred = np.multiply((y_pred_class[:,1] > thresher[i]),1)
    
    tp = sum(y_pred*y_true)
    tn = sum((1-y_pred)*(1-y_true))
    fn = sum((1-y_pred)*y_true)
    fp = sum(y_pred*(1-y_true))

    toter[i] = (fn+fp)/(len(y_true))
    fnr[i] = fn / [fn + tp]
    fpr[i] = fp / [fp + tn]
    reduc[i] = (tn + fn)/len(y_true)
    
sumerate = fpr + fnr
minsumthresh = np.argmin(sumerate)
minthresh = np.argmin(toter)

    

In [ ]:
plt.plot(thresher,fnr)
plt.plot(thresher,fpr)
plt.plot(thresher,fpr+fnr)
plt.plot(thresher,toter)
plt.legend(('FNR','FPR','Sum','Error Rate'))
plt.grid()
plt.ylim((0,.5))
plt.scatter(thresher[minsumthresh],sumerate[minsumthresh])
plt.scatter(thresher[minthresh],sumerate[minthresh])

optreduc = (sum(1-y_true))/len(y_true)

print('Best Sum Error is ' + str(round(sumerate[minsumthresh]*100,2)) + '% At threshold of ' + str(round(thresher[minsumthresh],2)))
print('Best Error is ' + str(round(toter[minthresh]*100,2)) + '% At threshold of ' + str(round(thresher[minthresh],2)))
print('Corresponds to FNR of ' + str(round(100*fnr[minthresh],2)) + '%')
print('Corresponds to FPR of ' + str(round(100*fpr[minthresh],2)) + '%')
print('Reduces by ' + str(round(100*reduc[minthresh],2)) + '%')
print('Optimal reduction is' + str(round(100*optreduc)) + '%')
print('Efficiency is ' + str(round(100*reduc[minthresh]/optreduc)))

In [ ]:
import cmocean 

Ebins = np.linspace(-5,3,51)
Hbins = np.linspace(0,4,41)
Cbins = np.linspace(0,1,31)

testE = np.sum(X_test[:,:25]*dFreq.T,axis=1)
testH = X_test[:,-2]
testC = X_test[:,-1]

Eloc = np.digitize(np.log10(testE),Ebins,right=True)
Hloc = np.digitize(testH,Hbins,right=True)
Cloc = np.digitize(testC,Cbins,right=True)

y_pred = np.multiply((y_pred_class[:,1] > thresher[minthresh]),1)
errmat = np.abs((y_true - y_pred))


accummat_error_EH = np.zeros((len(Ebins),len(Hbins)))
accummat_num_EH = np.zeros((len(Ebins),len(Hbins)))
accummat_numgo_EH = np.zeros((len(Ebins),len(Hbins)))
accummat_numno_EH = np.zeros((len(Ebins),len(Hbins)))


accummat_error_EC = np.zeros((len(Ebins),len(Cbins)))
accummat_num_EC = np.zeros((len(Ebins),len(Cbins)))
accummat_numgo_EC = np.zeros((len(Ebins),len(Cbins)))
accummat_numno_EC = np.zeros((len(Ebins),len(Cbins)))

accummat_error_CH = np.zeros((len(Cbins),len(Hbins)))
accummat_num_CH = np.zeros((len(Cbins),len(Hbins)))
accummat_numgo_CH = np.zeros((len(Cbins),len(Hbins)))
accummat_numno_CH = np.zeros((len(Cbins),len(Hbins)))

for i in range(len(Ebins)):
    for j in range(len(Hbins)):
        accummat_error_EH[i,j] = np.nanmean(errmat[(Eloc==i)& (Hloc==j)])
        accummat_num_EH[i,j] = len(errmat[(Eloc==i)& (Hloc==j)])
        accummat_numgo_EH[i,j] = np.sum(y_true[(Eloc==i)& (Hloc==j)])
        accummat_numno_EH[i,j] = np.sum(y_pred[(Eloc==i)& (Hloc==j)])
  
for i in range(len(Ebins)):
    for j in range(len(Cbins)):
        accummat_error_EC[i,j] = np.nanmean(errmat[(Eloc==i)& (Cloc==j)])
        accummat_num_EC[i,j] = len(errmat[(Eloc==i)& (Cloc==j)])
        accummat_numgo_EC[i,j] = np.sum(y_true[(Eloc==i)& (Cloc==j)])
        accummat_numno_EC[i,j] = np.sum(y_pred[(Eloc==i)& (Cloc==j)])


for i in range(len(Cbins)):
    for j in range(len(Hbins)):
        accummat_error_CH[i,j] = np.nanmean(errmat[(Cloc==i)& (Hloc==j)])
        accummat_num_CH[i,j] = len(errmat[(Cloc==i)& (Hloc==j)])
        accummat_numgo_CH[i,j] = np.sum(y_true[(Cloc==i)& (Hloc==j)])
        accummat_numno_CH[i,j] = np.sum(y_pred[(Cloc==i)& (Hloc==j)])


accummat_numgo_EH = accummat_numgo_EH / accummat_num_EH
accummat_numgo_EC = accummat_numgo_EC / accummat_num_EC
accummat_numgo_CH = accummat_numgo_CH / accummat_num_CH

accummat_numno_EH = accummat_numno_EH / accummat_num_EH
accummat_numno_EC = accummat_numno_EC / accummat_num_EC
accummat_numno_CH = accummat_numno_CH / accummat_num_CH


accummat_num_EH = accummat_num_EH / np.nansum(accummat_num_EH) 
accummat_num_EC = accummat_num_EC / np.nansum(accummat_num_EC)
accummat_num_CH = accummat_num_CH / np.nansum(accummat_num_CH)




In [ ]:
fig = plt.figure(num=None, figsize=(14, 10), dpi=80, facecolor='w', edgecolor='k');

plt.subplot(231)
plt.pcolormesh(Hbins,Ebins,accummat_error_EH,cmap='cmo.balance');
plt.colorbar()
plt.title('Avg. SAE');
plt.xlabel('Ice Thickness')
plt.ylabel('log Energy')
# plt.clim([0,2])

plt.subplot(232)
plt.pcolormesh(Cbins,Ebins,accummat_error_EC,cmap='cmo.balance');
plt.colorbar()
plt.title('Avg. SAE');
plt.xlabel('Ice Conc')
plt.ylabel('log Energy')
# plt.clim([0,2])

plt.subplot(233)
plt.pcolormesh(Hbins,Cbins,accummat_error_CH,cmap='cmo.balance');
plt.colorbar()
plt.title('Avg. SAE');
plt.xlabel('Ice Thickness')
plt.ylabel('Ice Conc')
# plt.clim([0,2])

plt.subplot(234)
plotter = np.nanmean(accummat_error_EH,axis=1);
plt.plot(Ebins,plotter)
plotter = np.nansum(accummat_num_EH,axis=1)
plt.plot(Ebins,plotter)
plotter = np.nanmean(accummat_numgo_EH,axis=1)
plt.plot(Ebins,plotter)
plotter = np.nanmean(accummat_numno_EH,axis=1)
plt.plot(Ebins,plotter)
plt.title('SAE')
plt.xlabel('Log Energy (m)')
plt.ylim([0,1])
plt.xlim([-3,2])
plt.grid()

plt.subplot(235)
plotter = np.nanmean(accummat_error_EH[:,:],axis=0);
plt.plot(Hbins,plotter)
plotter = np.nansum(accummat_num_EH[:,:],axis=0)
plt.plot(Hbins,plotter)
plotter = np.nanmean(accummat_numgo_EH[:,:],axis=0)
plt.plot(Hbins,plotter)
plotter = np.nanmean(accummat_numno_EH[:,:],axis=0)
plt.plot(Hbins,plotter)
plt.title('SAE')
plt.xlabel('Ice Thickness (m)')
plt.ylim([0,1])
plt.legend(('Error','Number'))


plt.grid()

plt.subplot(236)
plotter = np.nanmean(accummat_error_CH[:,:],axis=1);
plt.plot(Cbins,plotter)
plotter = np.nansum(accummat_num_CH[:,:],axis=1)
plt.plot(Cbins,plotter)
plotter = np.nanmean(accummat_numgo_CH[:,:],axis=1)
plt.plot(Cbins,plotter)
plotter = np.nanmean(accummat_numno_CH[:,:],axis=1)

plt.plot(Cbins,plotter)
plt.title('SAE')
plt.xlabel('Ice Conc (m)')
plt.ylim([0,1])
plt.legend(('Error','Relative Number','Frac Running','Frac Running (pred)'))

plt.grid()

#fig.tight_layout()
plt.savefig('Fullnet-Error-withlast.pdf')

In [ ]:
import seaborn as sns
import pandas as pd

thresh_opt = thresher[minthresh]
y_pred = np.multiply((y_pred_class[:,1] > thresh_opt),1)


index = random.choices(np.arange(len(X_test)), k=1000) # Any type

WE = np.log10(np.sum(X_test[index,:25]*np.expand_dims(dFreq,axis=1).T,axis=2).T)
H = np.expand_dims(X_test[index,-2],axis=1)
C = np.expand_dims(X_test[index,-1],axis=1)
USE = np.expand_dims(y_pred[index],axis=1) # Whether the input leads to fracture

df = pd.DataFrame(np.concatenate((WE,H,USE),axis=1),columns=['Wave Energy','Ice Thickness','Usable'])

In [ ]:
g = sns.scatterplot(
    x='Ice Thickness',
    y='Wave Energy',
    data=df,    
    hue='Usable',
)
# both, hue and size are optional
sns.despine() # prettier layout

In [15]:
from scipy.io import loadmat,savemat
save_string = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/Data/Neural_Net_Data/6-hourly-2009/Classifier-results.mat'
mdic = {"Y_true": y_test[:,1], "Y_pred": y_pred_class[:,1], "X_test":X_test}
savemat(save_string,mdic)
